In [2]:
from Tool_Functions.cleaning_data import *
from Tool_Functions.comportment_reabo import * 
from Tool_Functions.join_data import * 

In [3]:
path_antoine = "/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/"

Nouveaux indicateurs : 
- Mois du réabonnement 
- Delai de réabonnement moyen pour cette promo comparé au délai de réabonnement moyen pour toutes les promos
- Trouver aussi un moyen de quantifier le poids de l’abonnement par rapport aux habitudes de fidélité d’un client : s’il s’abonne alors qu’il s’abonne jamais, la promo aura un poids plus important

•⁠  ⁠transformer la fidélité en un score et essayer de le croiser avec les types de promo
•⁠  ⁠⁠comparer les délais de réabonnements à ce qu’il se fait hors promotion 
•⁠  ⁠⁠on peut dans un premier temps se concentrer sur un seul type d’ODD (ex 15j TC)

In [66]:
def clean_nombres_abonnements(abonnements):
    """
    Convertit les valeurs de la colonne 'NOMBRE_ABONNEMENTS' en listes d'entiers.
    """
    try:
        return eval(abonnements)
    except:
        return []

In [71]:
def calculer_score_fidélité_v1(abonnements):
   
    # Si l'abonné ne s'est jamais abonné, retourner un score minimal
    if not abonnements:
        return 0
    
    # Calculer le score comme la somme des abonnements sur la période
    score = sum(abonnements)
    
    nombre_abonnements = len(abonnements)
    
    score_maximal = 13 * nombre_abonnements
    
    score_normalisé = (score / score_maximal) 
    
    return score_normalisé

In [72]:
def colonne_score_v1(data_path):
    df = file_to_dataframe(data_path + "fusion_table_v3.csv")
    df['NOMBRE_ABONNEMENTS'] = df['NOMBRE_ABONNEMENTS'].apply(clean_nombres_abonnements)
    df['SCORE_FIDELITE'] = df['NOMBRE_ABONNEMENTS'].apply(calculer_score_fidélité_v1)
    save_to_csv_file(df, path_antoine + 'fusion_table_score_v1.csv')
    return True

In [73]:
colonne_score_v1(path_antoine)

True

In [82]:
df = file_to_dataframe(path_antoine + 'fusion_table_score_v1.csv')

In [83]:
df

,ID_ABONNE,Autres_n_REABOS,ODD 15 jours EV+_n_REABOS,ODD 15 jours TC_n_REABOS,ODD 21 jours TC_n_REABOS,ODD 30 jours EV+_n_REABOS,ODD 30 jours TC_n_REABOS,ODD 7 jours autre que SG_n_REABOS,PAS_ODD_n_REABOS,Semaine genéreuse_n_REABOS,...,5,6,7,8,9,10,11,12,NOMBRE_ABONNEMENTS,SCORE_FIDELITE
0,50402221602,0,0,1,0,0,2,0,1,0,...,0,1,1,0,0,0,0,2,"[3, 1]",0.153846
1,50402223202,0,0,8,0,0,4,0,0,17,...,3,2,1,2,2,3,2,2,"[11, 9, 9]",0.743590
2,50402224003,0,0,3,0,0,0,0,0,4,...,1,1,0,0,0,0,0,1,[7],0.538462
3,50402230202,0,0,9,0,0,3,0,0,6,...,2,2,1,1,1,2,1,2,"[8, 10]",0.692308
4,50402230203,0,0,11,1,0,3,0,0,5,...,2,1,1,1,1,2,1,2,"[8, 12]",0.769231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
780791,53186931901,0,1,1,0,0,0,0,0,1,...,0,0,0,0,2,1,0,0,[3],0.230769
780792,53186937201,0,1,1,0,0,0,0,0,1,...,0,0,0,0,2,1,0,0,[3],0.230769
780793,53186971001,0,1,1,0,0,0,0,0,1,...,0,0,0,0,2,1,0,0,[3],0.230769
780794,53186977101,0,1,1,0,0,0,0,0,1,...,0,0,0,0,2,1,0,0,[3],0.230769


In [92]:
def calculer_score_fidélité_v2(abonnements):
   
    # Si l'abonné ne s'est jamais abonné, retourner un score minimal
    if not abonnements:
        return 0
    
    # Calculer le score comme la somme des abonnements sur la période
    score = sum(abonnements)
    
    nombre_abonnements = len(abonnements)
    
    score_maximal = 13 * nombre_abonnements
    
    score_normalisé = (score / score_maximal) 
    
    poids_fidélité_récente = 0.6
    
    # Dernier abonnement
    dernier_abonnement = abonnements[-1]
    
    # Score ajusté en fonction du nombre de chiffres dans la liste des abonnements
    score_ajusté = score_normalisé * (1 + 0.1 * (nombre_abonnements - 1))
    
    # Score final
    score_final = (score_ajusté * poids_fidélité_récente) + (dernier_abonnement * (1 - poids_fidélité_récente))
    
    return score_final

In [103]:
def colonne_score_v2(data_path):
    df = file_to_dataframe(data_path + "fusion_table_score_v1.csv")
    df['NOMBRE_ABONNEMENTS'] = df['NOMBRE_ABONNEMENTS'].apply(clean_nombres_abonnements)
    df['SCORE_FIDELITE_2'] = df['NOMBRE_ABONNEMENTS'].apply(calculer_score_fidélité_v2)
    save_to_csv_file(df, path_antoine + 'fusion_table_score_v2.csv')
    return True

In [104]:
colonne_score_v2(path_antoine)

True

In [105]:
df2 = file_to_dataframe(path_antoine + 'fusion_table_score_v2.csv')

In [106]:
df2

,ID_ABONNE,Autres_n_REABOS,ODD 15 jours EV+_n_REABOS,ODD 15 jours TC_n_REABOS,ODD 21 jours TC_n_REABOS,ODD 30 jours EV+_n_REABOS,ODD 30 jours TC_n_REABOS,ODD 7 jours autre que SG_n_REABOS,PAS_ODD_n_REABOS,Semaine genéreuse_n_REABOS,...,6,7,8,9,10,11,12,NOMBRE_ABONNEMENTS,SCORE_FIDELITE,SCORE_FIDELITE_2
0,50402221602,0,0,1,0,0,2,0,1,0,...,1,1,0,0,0,0,2,"[3, 1]",0.153846,0.501538
1,50402223202,0,0,8,0,0,4,0,0,17,...,2,1,2,2,3,2,2,"[11, 9, 9]",0.743590,4.135385
2,50402224003,0,0,3,0,0,0,0,0,4,...,1,0,0,0,0,0,1,[7],0.538462,3.123077
3,50402230202,0,0,9,0,0,3,0,0,6,...,2,1,1,1,2,1,2,"[8, 10]",0.692308,4.456923
4,50402230203,0,0,11,1,0,3,0,0,5,...,1,1,1,1,2,1,2,"[8, 12]",0.769231,5.307692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
780791,53186931901,0,1,1,0,0,0,0,0,1,...,0,0,0,2,1,0,0,[3],0.230769,1.338462
780792,53186937201,0,1,1,0,0,0,0,0,1,...,0,0,0,2,1,0,0,[3],0.230769,1.338462
780793,53186971001,0,1,1,0,0,0,0,0,1,...,0,0,0,2,1,0,0,[3],0.230769,1.338462
780794,53186977101,0,1,1,0,0,0,0,0,1,...,0,0,0,2,1,0,0,[3],0.230769,1.338462
